# Introduction to SPEA2

The Strength Pareto Evolutionary Algorithm 2 (SPEA2) is an advanced evolutionary computation technique designed to solve multi-objective optimization problems. Multi-objective optimization involves optimizing two or more conflicting objectives simultaneously, a common challenge in various fields of engineering, economics, and logistics. SPEA2 stands out by effectively identifying a set of optimal solutions known as the Pareto front, which represents the trade-offs between the objectives where no single solution is superior to the others across all objectives.

### Core Principles of SPEA2

SPEA2 operates on a population of potential solutions, evolving them over generations through selection, crossover, and mutation operations. A unique aspect of SPEA2 is its use of a strength value to measure the dominance of each solution within the population. This strength value quantifies how many solutions a particular solution dominates, playing a key role in fitness assignment. Furthermore, SPEA2 introduces a density estimate to differentiate between solutions with similar strength values, enhancing the algorithm's ability to maintain diversity among the optimal solutions.

### Fitness Assignment and Environmental Selection

In SPEA2, fitness assignment combines the strength of a solution with its density estimate, ensuring that solutions that dominate many others but are not crowded (i.e., unique in the solution space) are favored. The algorithm also maintains an external archive, a secondary population that stores the best solutions found during the search process. This archive helps in preserving diversity and ensures that the evolutionary process does not lose potentially optimal solutions.

### Variation Operators

The application of crossover and mutation operators in SPEA2 encourages the exploration of the solution space. Crossover combines parts of two solutions to create offspring, potentially inheriting the strengths of both parents. Mutation introduces random changes to a solution, exploring new areas of the solution space. These operators are key for generating diverse solutions and avoiding premature convergence to suboptimal regions of the solution space.

### Conclusion and Application

SPEA2's ability to balance the exploration and exploitation of the solution space, along with its sophisticated mechanisms for maintaining diversity and handling multiple objectives, makes it a powerful tool for multi-objective optimization. Its applications range from engineering design to resource management, where decision-makers are faced with the challenge of making optimal choices in the presence of conflicting objectives.

## Optimizing a Wind Farm Design with SPEA2

In this project, we will apply the Strength Pareto Evolutionary Algorithm 2 (SPEA2) to address a practical and highly relevant problem: optimizing the design of a small wind farm. Our objective will be twofold: to maximize the total power output while minimizing the installation costs. These goals are naturally conflicting, as achieving higher energy production typically will involve higher costs due to the need for more or better-quality turbines and their strategic placement to capture wind optimally.

The challenge in wind farm design will not just lie in the placement and selection of turbines but also in managing the constraints imposed by land use, environmental impact, and budgetary limits. Each turbine's position will affect the overall efficiency of the wind farm due to wake effects, where turbines downstream of others experience reduced wind speed, thus lowering their power output.

Our scenario will consider several decision variables, including the number and types of turbines and their geographical positions within the farm. By applying SPEA2, we will seek to explore a range of optimal solutions that demonstrate the trade-offs between power output and cost, providing valuable insights for stakeholders to make informed decisions based on their priorities.



**Decision Variables:**
- $x_{ij}$: Binary variable indicating whether turbine type $j$ is placed at position $i$, where $i \in \{1, 2, \ldots, N\}$ represents possible positions, and $j \in \{1, 2, \ldots, T\}$ represents turbine types.

**Parameters:**
- $P_{ij}$: Power output of turbine type $j$ at position $i$.
- $C_{j}$: Installation cost of turbine type $j$.
- $D_{ij}$: Distance between turbine position $i$ and all other turbines, affecting wake loss.
- $B$: Total budget available for installation.
- $N$: Number of potential positions for turbines.
- $T$: Number of turbine types.
- $MinDist$: Minimum required distance between any two turbines to avoid excessive wake effect.

**Objective Functions:**
1. Maximize Total Power Output: 
   $$Z_1 = \sum_{i=1}^{N}\sum_{j=1}^{T} P_{ij} \cdot x_{ij}$$
2. Minimize Installation Cost: 
   $$Z_2 = \sum_{j=1}^{T} C_{j} \cdot x_{ij}$$

**Constraints:**
1. Turbine Positioning Constraint (Only one turbine can be placed at each position):
   $$\sum_{j=1}^{T} x_{ij} \leq 1, \quad \forall i \in \{1, 2, \ldots, N\}$$
2. Budget Constraint (Total cost must not exceed available budget):
   $$\sum_{i=1}^{N}\sum_{j=1}^{T} C_{j} \cdot x_{ij} \leq B$$
3. Distance Constraint (To mitigate wake effect, turbines must be placed at a minimum distance from each other):
   $$D_{ij} \geq MinDist, \quad \forall i \neq j, \; x_{ij}, x_{jk} = 1$$


In [1]:
import numpy as np
import random

num_turbines = 10
possible_turbine_types = ['TypeA', 'TypeB', 'TypeC']  # Simplified turbine types
max_generations = 100

def initialize_population(population_size=50):
    population = []
    for _ in range(population_size):
        turbine_positions = [(random.uniform(0, 1000), random.uniform(0, 1000)) for _ in range(num_turbines)]
        turbine_types = [random.choice(possible_turbine_types) for _ in range(num_turbines)]
        solution = {'positions': turbine_positions, 'types': turbine_types, 'objectives': (0, 0)}
        population.append(solution)
    return population

def calculate_objectives(solution):
    power_output = sum([random.uniform(1, 2) for _ in solution['positions']])  # Simplified power output
    installation_cost = sum([random.uniform(100, 200) for _ in solution['types']])  # Simplified cost
    return power_output, installation_cost

def crossover(parent1, parent2):
    crossover_point = random.randint(1, num_turbines-1)
    offspring_positions = parent1['positions'][:crossover_point] + parent2['positions'][crossover_point:]
    offspring_types = parent1['types'][:crossover_point] + parent2['types'][crossover_point:]
    return {'positions': offspring_positions, 'types': offspring_types, 'objectives': (0, 0)}

def mutation(solution, mutation_rate=0.1):
    for i in range(num_turbines):
        if random.random() < mutation_rate:
            solution['positions'][i] = (random.uniform(0, 1000), random.uniform(0, 1000))
            solution['types'][i] = random.choice(possible_turbine_types)
    return solution

def spea2(population_size=50, max_generations=100):
    population = initialize_population(population_size)
    archive = []

    for generation in range(max_generations):
        for solution in population:
            solution['objectives'] = calculate_objectives(solution)
        
        # Simplified selection process for demonstration
        selected_indices = random.sample(range(len(population)), population_size // 2)
        archive = [population[i] for i in selected_indices]

        offspring_population = [mutation(crossover(population[random.randint(0, population_size // 2 - 1)],
                                                   population[random.randint(0, population_size // 2 - 1)]))
                                for _ in range(population_size // 2)]
        population = archive + offspring_population

    return archive

# Generate results
spea2_results = spea2()

# Display simplified results
for i, solution in enumerate(spea2_results):
    power_output, installation_cost = solution['objectives']
    print(f"Solution {i+1}: Power Output = {power_output:.2f}, Installation Cost = {installation_cost:.2f}")


Solution 1: Power Output = 15.50, Installation Cost = 1523.81
Solution 2: Power Output = 15.54, Installation Cost = 1552.51
Solution 3: Power Output = 15.31, Installation Cost = 1548.49
Solution 4: Power Output = 15.62, Installation Cost = 1656.15
Solution 5: Power Output = 15.52, Installation Cost = 1461.39
Solution 6: Power Output = 15.91, Installation Cost = 1394.23
Solution 7: Power Output = 16.01, Installation Cost = 1606.05
Solution 8: Power Output = 13.42, Installation Cost = 1476.72
Solution 9: Power Output = 15.08, Installation Cost = 1531.74
Solution 10: Power Output = 13.89, Installation Cost = 1509.42
Solution 11: Power Output = 14.27, Installation Cost = 1301.53
Solution 12: Power Output = 14.30, Installation Cost = 1505.34
Solution 13: Power Output = 16.03, Installation Cost = 1550.03
Solution 14: Power Output = 15.36, Installation Cost = 1434.50
Solution 15: Power Output = 15.45, Installation Cost = 1362.48
Solution 16: Power Output = 12.81, Installation Cost = 1411.22
S